In [2]:
import locale
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util
# import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd

C:\Users\Bouayad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/hamzafarooq/maven-mlsystem-design-cohort-1/main/data/paris_02_11_23.csv')

In [4]:
import sweetviz as sv

report = sv.analyze(df)
report.show_html('report.html')


[Summarizing dataframe]                      |          | [  0%]   00:00 -> (? left)

Feature: date                                |█         | [ 11%]   00:00 -> (00:06 left)C:\Users\Bouayad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sweetviz\from_profiling_pandas.py:65: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(keys):
Feature: review_rating                       |█▌        | [ 16%]   00:00 -> (00:05 left)C:\Users\Bouayad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sweetviz\from_profiling_pandas.py:65: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(keys):
Feature: title                               |██        | [ 21%]   00:00 -> (00:02 left)C:\Users

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [5]:
df.head()

,review_id,date,review_rating,title,text,votes,url,language,platform,author_id,author_name,author_username,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,/ShowUserReviews-g187147-d207742-r864290614-In...,en,MOBILE,E488EBBA1F82F16BF878FE274C735941,Anna J,AnnaJ250,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,/ShowUserReviews-g187147-d207742-r864049819-In...,en,MOBILE,4A830AD8B128F60AC02E83D6B6A530F7,QATAR2007,QATAR2007,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,/ShowUserReviews-g187147-d207742-r863952022-In...,en,OTHER,AA2958C3E083861E81EEC085671BAA5B,aji1376,aji1376,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,/ShowUserReviews-g187147-d207742-r863793066-In...,en,MOBILE,DE4AB96DA3E104846D6D6423C2DAA4C8,jelinc2016,jelinc2016,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,/ShowUserReviews-g187147-d207742-r863631994-In...,en,MOBILE,DE02D713F209AEC684DC6108509E6912,VikaasK,VikaasK,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [6]:
#afficher les values uniques de la colonne 'language' et la fréquence (en pourcentage) de chaque valeur
df['language'].value_counts(normalize=True)

language
en    1.0
Name: proportion, dtype: float64

In [7]:
#Lister les colonnes du dataframe
df.columns

Index(['review_id', 'date', 'review_rating', 'title', 'text', 'votes', 'url',
       'language', 'platform', 'author_id', 'author_name', 'author_username',
       'name', 'id', 'description', 'rating', 'rating_count', 'features'],
      dtype='object')

In [8]:
#les valeurs de la colonne plateform
df['platform'].value_counts(normalize=True)

platform
OTHER     0.669060
MOBILE    0.330592
TABLET    0.000349
Name: proportion, dtype: float64

In [9]:
#afficher les valeurs de la colonne features des 10 premières hotels (sur la base de la colonne 'id')
df['features'].head(10)

0    ['roomFeatures_air conditioning', 'roomFeature...
1    ['roomFeatures_air conditioning', 'roomFeature...
2    ['roomFeatures_air conditioning', 'roomFeature...
3    ['roomFeatures_air conditioning', 'roomFeature...
4    ['roomFeatures_air conditioning', 'roomFeature...
5    ['roomFeatures_air conditioning', 'roomFeature...
6    ['roomFeatures_air conditioning', 'roomFeature...
7    ['roomFeatures_air conditioning', 'roomFeature...
8    ['roomFeatures_air conditioning', 'roomFeature...
9    ['roomFeatures_air conditioning', 'roomFeature...
Name: features, dtype: object

In [10]:
#pour chaque colonne affiche son type 
df.dtypes

review_id            int64
date                object
review_rating        int64
title               object
text                object
votes                int64
url                 object
language            object
platform            object
author_id           object
author_name         object
author_username     object
name                object
id                   int64
description         object
rating             float64
rating_count       float64
features            object
dtype: object

In [11]:
#convertir la colonne 'date' en type datetime
df['date'] = pd.to_datetime(df['date'])


In [12]:
#convertir la colonne rating en type float
df['rating'] = df['rating'].astype(float)

In [13]:
#créer une liste avec les colonnes à supprimer (url, language, platform, author_id, author_name, author_username) et créer une fonction qui supprime les colonne mentionnées dans la liste
to_drop = ['url', 'language', 'platform', 'author_id', 'author_name', 'author_username']
def drop_cols(df, cols):
    df.drop(columns=cols, inplace=True)
drop_cols(df, to_drop)

In [14]:
#nombre de doublons sur la base de la colonne 'review_id'
df.duplicated(subset=['review_id']).sum()





16704

In [15]:
#df.drop_duplicates(subset=['review_id'], inplace=True)

In [16]:
#valeur min et max de la colonne 'date'
df['date'].min(), df['date'].max()

(Timestamp('2013-06-19 00:00:00'), Timestamp('2022-10-15 00:00:00'))

In [17]:
#répartition du nombre de commentaires (review_id) par an (sur la base de la colonne 'date') et classe le résultat par ordre croissant par an
df['date'].dt.year.value_counts().sort_index()

date
2013      28
2014      36
2015     210
2016     818
2017    1670
2018    3412
2019    8120
2020    2494
2021    2738
2022    9168
Name: count, dtype: int64

In [18]:
#Supprime les lignes où la date du review est antérieur à 2017 sur la base de la colonne 'date'
df = df[df['date'].dt.year >= 2017]

In [19]:
df.describe()

,review_id,date,review_rating,votes,id,rating,rating_count
count,2.760200e+04,27602,27602.000000,27602.000000,2.760200e+04,13830.000000,13830.000000
mean,7.394518e+08,2020-07-27 20:36:50.970219776,4.411057,0.794725,2.486359e+06,4.392842,1132.042372
min,4.483076e+08,2017-01-01 00:00:00,1.000000,-1.000000,1.887380e+05,4.000000,58.000000
25%,6.749037e+08,2019-05-18 00:00:00,4.000000,0.000000,2.076820e+05,4.000000,450.000000
50%,7.403564e+08,2020-01-20 00:00:00,5.000000,0.000000,2.335700e+05,4.500000,726.000000
75%,8.367400e+08,2022-04-30 00:00:00,5.000000,1.000000,7.773470e+05,4.500000,1440.000000
max,8.646788e+08,2022-10-15 00:00:00,5.000000,40.000000,2.404224e+07,5.000000,4438.000000
std,1.043685e+08,NaN,1.030286,1.137274,4.793415e+06,0.270275,1002.808093


In [20]:
#nombre de review_id
df['review_id'].count()

27602

In [21]:
#nombre de review_id uniques
df['review_id'].nunique()

11612

In [22]:
#supprimer les reviews doublons
df.drop_duplicates(subset=['review_id'], inplace=True)

In [23]:
df.describe()

,review_id,date,review_rating,votes,id,rating,rating_count
count,1.161200e+04,11612,11612.000000,11612.000000,1.161200e+04,5699.000000,5699.000000
mean,7.365495e+08,2020-07-10 23:53:33.089906944,4.385377,0.799087,2.569871e+06,4.383664,1106.119846
min,4.483076e+08,2017-01-01 00:00:00,1.000000,-1.000000,1.887380e+05,4.000000,58.000000
25%,6.699155e+08,2019-04-28 00:00:00,4.000000,0.000000,2.077430e+05,4.000000,455.000000
50%,7.366211e+08,2020-01-03 00:00:00,5.000000,0.000000,2.409270e+05,4.500000,881.000000
75%,8.365374e+08,2022-04-29 00:00:00,5.000000,1.000000,1.229799e+06,4.500000,1440.000000
max,8.646788e+08,2022-10-15 00:00:00,5.000000,40.000000,2.404224e+07,5.000000,4438.000000
std,1.045308e+08,NaN,1.047426,1.148828,4.854813e+06,0.286703,908.408889


In [24]:
df.head(30)

,review_id,date,review_rating,title,text,votes,name,id,description,rating,rating_count,features
0,864290614,2022-10-12,1,A large impersonal place with an on time check...,"If you are looking for a huge, grand hotel exp...",1,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
1,864049819,2022-10-10,4,Good hotel with rude waiter,We went to this hotel just this month\nWe have...,1,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
2,863952022,2022-10-10,5,Fantastic,"Absolutely top-notch. Room, service, bed, pill...",0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
3,863793066,2022-10-09,4,"Amidst the chaos of Fashion week, their servic...",We stayed during the Paris Fashion Week Chaos....,0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
4,863631994,2022-10-08,2,Not worth the effort or money,This hotel is not worth the effort or the pric...,0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
5,862073785,2022-09-26,4,Not quite up to Intercontinental standards,We had a one night stay prior to taking the Eu...,0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
6,861021031,2022-09-20,5,Luxury in the heart of the city,We were very impressed with the intercontinent...,0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
7,860421664,2022-09-16,5,Beautiful,Visited for my birthday and everything was jus...,0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
8,856124006,2022-08-24,5,Great old fashioned quality service,All I can say is we found good old fashioned f...,1,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."
9,856016947,2022-08-23,5,Best location and platform to visit Paris,Our family of 4 plus a friend enjoyed the grea...,0,InterContinental Paris - Le Grand,207742,"The InterContinental Paris Le Grand, opened du...",4.5,3517.0,"['roomFeatures_air conditioning', 'roomFeature..."


In [25]:
#convertir df en un fichier csv et le sauvegarder dans le répertoire courant sous le nom de "data_cleaned.csv"
df.to_csv('data_cleaned.csv', index=False)